In [1]:
#!wget https://arxiv.org/e-print/2004.04818
#!wget https://mirrors.ctan.org/obsolete/macros/latex/contrib/revtex4-1.zip
#!unzip revtex4-1.zip
#!cd revtex4-1/
#run in terminal
#find . -maxdepth 1 -type f -name "*.dtx" -exec  tex "{}" \;

In [115]:
from sys import argv
from collections import defaultdict
from nltk.tokenize import mwe
from nltk import sent_tokenize
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#from pudb import set_trace
argv=[]
argv.append('')
argv.append('sound1.tex')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [116]:
def clean_display_latex(tex):
    import re
    resp = re.sub(r'{\\rm ','{',tex)
    resp = re.sub(r'\\left\(','(',resp)
    resp = re.sub(r'\\right\)',')',resp)
    resp = re.sub(r'{\\it ','{',resp)
    return resp

In [117]:
class Trie(dict):
    #https://github.com/nltk/nltk/blob/develop/nltk/collections.py
    """A Trie implementation for strings"""

    LEAF = True

    def __init__(self, strings=None):
        """Builds a Trie object, which is built around a ``dict``

        If ``strings`` is provided, it will add the ``strings``, which
        consist of a ``list`` of ``strings``, to the Trie.
        Otherwise, it'll construct an empty Trie.

        :param strings: List of strings to insert into the trie
            (Default is ``None``)
        :type strings: list(str)

        """
        super(Trie, self).__init__()
        if strings:
            for string in strings:
                self.insert(string)

    def insert(self, string):
        if len(string):
            self[string[0]].insert(string[1:])
        else:
            if not self[Trie.LEAF]:
                self[Trie.LEAF] = 1
            else:
                self[Trie.LEAF]+=1


    def __missing__(self, key):
        self[key] = Trie()
        return self[key]


In [118]:
def add_new_token(string):
    tokenizer.add_mwe(r"{}".format(string))


def manual_entries(pattern, data):
    resp = re.findall(pattern, data)
    if resp:
        for match in resp:
            add_new_token(match)


def print_symbols_not_used():
    for k, v in symbols_use_count.items():
        if v == 0:
            print(k)


def read_file(path, f_name):
    with open("{}/{}".format(path, f_name), "r", encoding="ISO-8859-1") as f:
        data = f.read()
    return data


def symbols_not_accounted_for(seen_count):
    for item in sorted(dct.items(), key=lambda x: -x[1]):
        if item[1] > seen_count:
            print(item[0])

#will be used to expand simple new commands when tokens are not recognized.
def simple_replace(s):
    #works, needs more testing
    match = [x for x in re.findall(r"\\newcommand{(.*?)}{(.*?)}",s)]
    commands = {}
    if match:
        for tpl in match:
            commands[tpl[0]] = lambda x: x.replace(tpl[0],tpl[1])
    return commands

'''
for newcmd in re.findall(r'\\newcommand.*',data):
    dct.update(simple_replace(newcmd))
'''
""" \\newcommand{\\ri}{\\right} -> dct['\\ri']('hello \\ri') -> 'hello \\right' """


symbols = list()
dct = defaultdict(int)
tokenizer = mwe.MWETokenizer(separator="")
symbols_use_count = defaultdict(int)

path = "."
data = [x.replace("\n", "") for x in read_file(path, "latex_math.txt").splitlines()]
symbols.extend(data)

#for now only single expressions
path = "."
data = [x.replace("\n", "") for x in read_file(path, "user_defined_symbols_expressions.txt").splitlines()]
symbols.extend(data)


data = [x.replace("\n", "") for x in read_file(path, "latex_misc.txt").splitlines()]
symbols.extend(data)

data = [x.replace("\n", "") for x in read_file(path, "english_vocab.txt").splitlines()]
symbols.extend(data)
symbols = list(set(symbols))
symbols = sorted(symbols, key=lambda x: -len(x))
for symbol in symbols:
    add_new_token(symbol)

file_data = read_file(".", argv[1])
file_data = clean_display_latex(file_data)
def use_package(s):
    import re
    match = re.match(r"(?P<cmd>\\usepackage)(?P<package>{[a-z]+(,[a-z]+)*})?", s)
    if match:
        cmd = match.group("cmd")
        packages = match.group("package")
        if packages:
            packages = [x.strip() for x in re.split("{|}|,", packages) if x.strip()]
            if packages:
                return {cmd: packages}


def math_mode(s):
    import re
    match = re.findall(r"[^\$](\$\$.*?\$\$)[^\$]", s, re.DOTALL)
    single_line = re.findall(r"[^\$](\$.*?\$)[^\$]", s)
    match.extend(single_line)
    return match


def balanced(start, s, left_symbol=r"{", right_symbol=r"}"):
    import re
    matched = []
    # the pattern should include one instance of the left symbol so that balanced=-1
    # if left == right then +- counting fails
    # begin syntax handled automatically, so there is no need to include left_right symbols
    s = re.sub(r'\\newcommand.*','',file_data)
    if "\\begin" in start:
        match = re.findall(r"\\begin{(.*?)}", start)
        if match:
            match = match[0]
            # print(match)
            left_symbol = f"""\\begin{{{match}}}"""
            right_symbol = f"""\\end{{{match}}}"""
    print(left_symbol)
    if left_symbol != right_symbol:
        current_offset = 0
        balanced = 0
        match = re.finditer(r"{}".format(start), s)
        # print(match)
        for m in match:
            if m:
                # print(m)
                start_offset = m.start()
                current_offset = m.end() + 1
                balanced = -1
                while balanced != 0:
                    if (
                        s[current_offset : current_offset + len(right_symbol)]
                        == right_symbol
                    ):
                        balanced += 1
                    if (
                        s[current_offset : current_offset + len(left_symbol)]
                        == left_symbol
                    ):
                        print("-1")
                        balanced -= 1
                    current_offset += 1
                matched.append(s[start_offset:current_offset])
                start_offset = current_offset
    return matched



In [120]:
sents = nltk.sent_tokenize(file_data)

In [121]:
from time import sleep
from collections import defaultdict
data_dict = defaultdict(list)
nltk_data = list()
titles = balanced(r"\\title{",file_data)
if titles:
    for title in titles:
        data_dict['title'].append(title)
        file_data = re.sub(re.escape(title),'',file_data)
        
authors = balanced(r"\\author{",file_data)
if authors:
    for author in authors:
        data_dict['author'].append(author)
        file_data = re.sub(re.escape(author),'',file_data)
        
affiliations = balanced(r"\\affiliation{",file_data)
if affiliations:
    for affiliation in affiliations:
        data_dict['affiliation'].append(affiliation)
        file_data = re.sub(re.escape(affiliation),'',file_data)
        
equations = balanced(r"\\begin{equation}",file_data)
if equations:
    for equation in equations:
        data_dict['equation'].append(equation)
        for equation in equations:
            file_data = re.sub(re.escape(equation),'',file_data,re.MULTILINE)
        
abstracts = balanced(r"\\begin{abstract}",file_data)
if abstracts:
    for abstract in abstracts:
        data_dict['abstract'].append(abstract)
        nltk_data.append(abstract)
        file_data = re.sub(re.escape(abstract),'',file_data)
        

figures = balanced(r"\\begin{figure}",file_data)
if figures:
    for figure in figures:
        data_dict['figure'].append(figure)
        file_data = re.sub(re.escape(figure),'',file_data)
        
bibitems = re.findall(r'\\bibitem{.*?}.*?\n\n',file_data,re.DOTALL)
if bibitems:
    for bibitem in bibitems:
        data_dict['bibitem'].append(bibitem)
        file_data = re.sub(re.escape(bibitem),'',file_data)

{
{
{
\begin{equation}
\begin{abstract}
\begin{figure}


In [122]:
"""label='\\begin{abstract}'
code_gen=f'''{label}s = balanced(r"\\{label}",file_data)
if {label}s:
    for {label} in {label}s:
        data_dict['{label}'].append({label})
        file_data = re.sub(re.escape({label}),'',file_data)'''

print(code_gen.replace('\\','\\\\'))"""

'label=\'\\begin{abstract}\'\ncode_gen=f\'\'\'{label}s = balanced(r"\\{label}",file_data)\nif {label}s:\n    for {label} in {label}s:\n        data_dict[\'{label}\'].append({label})\n        file_data = re.sub(re.escape({label}),\'\',file_data)\'\'\'\n\nprint(code_gen.replace(\'\\\',\'\\\\\'))'

In [123]:
re.findall('begin{.*?}',file_data)

['begin{document}', 'begin{thebibliography}']

In [124]:
for ix,line in enumerate(file_data.splitlines()):
    #fd = line
    if '\\usepackage' in line:
        resp = use_package(line)
        if resp:
            for items in resp.values():
                for item in items:
                    data_dict['usepackage'].append(item)
            continue
    if '\\documentclass\[' in line:
        data_dict['documentclass'].append(line)
        continue
        
    if '%' in line:
        resp = False
        resp = re.findall('%.*',line)
        if resp:
            data_dict['comment'].append(line)
        continue
        
    
    else:
        try:
            line = re.sub(r'\\begin{document}|\\raggedbottom|\\maketitle|end{abstract}| \
                          \\section{.*?}|end{.*?}|\\begin{thebibliography}','',line)
            line = re.sub(r'\\noindent ',' ',line)
            if line.strip():
                nltk_data.append(line)
        except Exception as e:
            print(e)
    

In [125]:
nltk_data_v1 = '\n'.join(nltk_data)


In [126]:
sents  = nltk.sent_tokenize(nltk_data_v1)

In [127]:
for sent in sents:
    print(sent)
    sleep(3)
    print('\n')

\begin{abstract}
Two dimensionless fundamental physical constants, the fine structure constant $\alpha$ and the proton-to-electron mass ratio $\frac{m_p}{m_e}$ are attributed a particular importance from the point of view of nuclear synthesis, formation of heavy elements, planets, and life-supporting structures.


Here, we show that a combination of these two constants results in a new dimensionless constant which provides the upper bound for the speed of sound in condensed phases, $v_u$.


We find that $\frac{v_u}{c}=\alpha(\frac{m_e}{2m_p})^{\frac{1}{2}}$, where $c$ is the speed of light in vacuum.


KeyboardInterrupt: 

In [74]:
#sorted([x for x in nltk.FreqDist(nltk.ngrams(nltk.wordpunct_tokenize(nltk_data_v1),2)).items() if x[1] > 5],key=lambda x:-x[1])

In [70]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from time import sleep
from sys import argv


document = nltk_data_v1


tokenizer = Tokenizer("english")
stemmer = Stemmer("english")
parser = PlaintextParser(document, tokenizer)


stop_words = [x for x in get_stop_words("english") if len(x) > 1]
summarizer = LsaSummarizer(stemmer)
summarizer.stop_words = stop_words

count = 0
summy_sents =[]
word_count = 0
for sent in summarizer(parser.document, len(document.splitlines())/1):
    word_count += len(sent.words)
    count+=1
    #sleep(5)
    #print()


In [129]:
#The goal is to extract all of the symbols, then find the meaning of each symbol in the tex file.
[clean_display_latex(x) for x in 
 set(re.findall(r'\$.*?\$',nltk_data_v1)) 
 if not re.findall('frac|=',str(x)) 
 and re.findall('\\\\',str(x))
]

['$\\hbar\\omega_{D}$',
 '$\\hbar$',
 '$\\alpha$',
 '$\\alpha c$',
 '$\\omega_{D}$',
 '$\\rho$']

In [ ]:
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder,TrigramAssocMeasures

#from nltk.collocations.py source
def demo(scorer=None, compare_scorer=None, text=nltk_data_v1):

    from nltk.metrics import (
        BigramAssocMeasures,
        spearman_correlation,
        ranks_from_scores,
    )

    if scorer is None:
        scorer = BigramAssocMeasures.likelihood_ratio
    if compare_scorer is None:
        compare_scorer = BigramAssocMeasures.raw_freq

    from nltk.corpus import stopwords, webtext

    ignored_words = stopwords.words("english")
    word_filter = lambda w: len(w) < 3 or w.lower() in ignored_words

    if 1 == 1:
        words = [word.lower() for word in nltk.word_tokenize(text)]
        
        cf = BigramCollocationFinder.from_words(words)
        cf.apply_freq_filter(3)
        cf.apply_word_filter(word_filter)

        corr = spearman_correlation(
            ranks_from_scores(cf.score_ngrams(scorer)),
            ranks_from_scores(cf.score_ngrams(compare_scorer)),
        )
        #print(file)
        print("\t", [" ".join(tup) for tup in cf.nbest(scorer, 25)])
        print("\t Correlation to %s: %0.4f" % (compare_scorer.__name__, corr))



In [173]:
#source from nltk collocations.py file
from nltk.corpus import stopwords
from nltk.metrics import (
        BigramAssocMeasures,
        spearman_correlation,
        ranks_from_scores,
    )
ignored_words = stopwords.words("english")

word_filter = lambda w: len(w) < 3 or w.lower() in ignored_words
words = [word.lower() for word in nltk.word_tokenize(nltk_data_v1)]
words = [x for x in words if not word_filter(x)]
words = [x for x in words if x !='\\ref']
scorer = TrigramAssocMeasures.likelihood_ratio
tgcf = TrigramCollocationFinder.from_words(words)
compare_scorer = TrigramAssocMeasures.raw_freq
corr = spearman_correlation(
            ranks_from_scores(tgcf.score_ngrams(scorer)),
            ranks_from_scores(tgcf.score_ngrams(compare_scorer)),
        )
        
print("\t", [" ".join(tup) for tup in tgcf.nbest(scorer, 15)])
print("\t Correlation to %s: %0.4f" % (compare_scorer.__name__, corr))


	 ['\\frac m_p m_e', 'fine structure constant', 'using a=a_ bohr', 'provides upper bound', 'upper bound terms', 'upper bound speed', 'arrive upper bound', 'upper bound sets', 'discussed upper bound', 'representing upper bound', 'new upper bound', 'upper bound v_u', 'upper bound v00', 'fundamental physical constants', 'entropy heat capacity']
	 Correlation to raw_freq: -0.6716
